In [101]:
import pandas as pd
import datetime
from threading import Thread, Lock

In [2]:
data_ = pd.read_csv('..\data\charts.csv')
data_.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [3]:
data = data_#.head(100000)

In [4]:
# Artist
df_artist = pd.DataFrame(data['artist'].unique(), columns=['name'])
df_artist = df_artist.assign(name=df_artist['name'].str.split(', ')).explode('name')
df_artist = pd.DataFrame(df_artist['name'].unique(), columns=['name'])
df_artist.reset_index(drop = True, inplace=True)
df_artist.insert(0, 'artist_id', range(1, 1 + len(df_artist)))

# Song
df_song = pd.DataFrame(data['title'].unique(), columns=['title'])
df_song.insert(0, 'song_id', range(1, 1 + len(df_song)))

# Trend
df_trend = pd.DataFrame(data['trend'].unique(), columns=['trend'])
df_trend.insert(0, 'trend_id', range(1, 1 + len(df_trend)))

# Day
df_day = pd.DataFrame(data['date'].unique(), columns=['date'])
df_day.insert(0, 'day_id', range(1, 1 + len(df_day)))

# Region
df_region = pd.DataFrame(data['region'].unique(), columns=['name'])
df_region.insert(0, 'region_id', range(1, 1 + len(df_region)))

# Category
df_category = pd.DataFrame(data['chart'].unique(), columns=['name'])
df_category.insert(0, 'category_id', range(1, 1 + len(df_category)))

In [5]:
artistDict = df_artist.set_index('name').T.to_dict('records')[0]
songDict = df_song.set_index('title').T.to_dict('records')[0]
trendDict = df_trend.set_index('trend').T.to_dict('records')[0]
dayDict = df_day.set_index('date').T.to_dict('records')[0]
regionDict = df_region.set_index('name').T.to_dict('records')[0]
categoryDict = df_category.set_index('name').T.to_dict('records')[0]

In [6]:
df = pd.DataFrame(columns = ['chart_id', 'position', 'song_id', 'day_id', 'region_id', 'category_id', 'trend_id', 'streams'])
df.to_csv('../data/df_chart.csv', mode='w', index = 'False')

In [85]:
start = 0 # enter the index of the next line to write (to continue enter (last chart_id)), for first time enter 0
end = 10000000-1 # last index to write (last value of chart_id will be end+1)
data = data_.loc[start:end]
data.reset_index(drop = True, inplace=True)
print(f"Length of data: {len(data)}")
print(data.head(10)["title"])

Length of data: 10000000
0        Chantaje (feat. Maluma)
1    Vente Pa' Ca (feat. Maluma)
2     Reggaetón Lento (Bailemos)
3                         Safari
4                    Shaky Shaky
5                    Traicionera
6        Cuando Se Pone a Bailar
7      Otra vez (feat. J Balvin)
8                   La Bicicleta
9         Dile Que Tu Me Quieres
Name: title, dtype: object


In [92]:
#dfs = []
batches = 100
batch = 10000
num = 0
df_chart_final = pd.DataFrame()
start = datetime.datetime.now().timestamp()
for j in range(batches):
    df_chart = pd.DataFrame(columns=[ 'title','rank', 'date', 'region', 'chart', 'trend', 'streams'])
    start = datetime.datetime.now().timestamp()
    for i in range(batch):
        
        df_row = data.iloc[num]
        title = songDict[df_row.iloc[0]]
        rank = df_row.iloc[1]
        date = dayDict[df_row.iloc[2]]
        region = regionDict[df_row.iloc[5]]
        chart = categoryDict[df_row.iloc[6]]
        trend = trendDict[df_row.iloc[7]]
        streams = df_row.iloc[8]
        row = [title, rank, date, region, chart, trend, streams]
        df_chart.loc[num] = row
    
        num +=1
        pass
    df_chart_final = pd.concat([df_chart_final, df_chart])
    
    print(f"Batch {j}/{batches} done [{datetime.datetime.now().timestamp()-start}]")

#print(df_chart)

Batch 0/100 done [12.827239036560059]
Batch 1/100 done [12.7189199924469]
Batch 2/100 done [12.631396055221558]
Batch 3/100 done [13.697654962539673]
Batch 4/100 done [12.729915857315063]
Batch 5/100 done [12.662709951400757]
Batch 6/100 done [13.174784898757935]
Batch 7/100 done [12.548977136611938]
Batch 8/100 done [12.818909883499146]
Batch 9/100 done [12.259067058563232]
Batch 10/100 done [13.652045965194702]
Batch 11/100 done [13.483829975128174]
Batch 12/100 done [13.254961967468262]
Batch 13/100 done [15.072728872299194]
Batch 14/100 done [14.327663898468018]
Batch 15/100 done [13.03382921218872]
Batch 16/100 done [13.8469398021698]
Batch 17/100 done [12.832465171813965]
Batch 18/100 done [12.623512029647827]
Batch 19/100 done [13.768102884292603]
Batch 20/100 done [13.793585062026978]
Batch 21/100 done [12.89687204360962]
Batch 22/100 done [12.838106870651245]
Batch 23/100 done [13.635195016860962]
Batch 24/100 done [13.382941961288452]
Batch 25/100 done [13.364902019500732]
Ba

In [93]:
#df_chart_final = pd.concat(dfs)
print(df_chart_final)

         title  rank   date  region  chart  trend   streams
0          1.0   1.0    1.0     1.0    1.0    1.0  253019.0
1          2.0   2.0    1.0     1.0    1.0    2.0  223988.0
2          3.0   3.0    1.0     1.0    1.0    3.0  210943.0
3          4.0   4.0    1.0     1.0    1.0    1.0  173865.0
4          5.0   5.0    1.0     1.0    1.0    2.0  153956.0
...        ...   ...    ...     ...    ...    ...       ...
999995  3154.0  92.0  157.0    16.0    1.0    3.0    3096.0
999996  2532.0  93.0  157.0    16.0    1.0    3.0    3094.0
999997   862.0  94.0  157.0    16.0    1.0    2.0    3088.0
999998     6.0  95.0  157.0    16.0    1.0    3.0    3068.0
999999  2496.0  96.0  157.0    16.0    1.0    3.0    3029.0

[1000000 rows x 7 columns]


In [99]:
print(data.iloc[999995:1000002]["streams"])

999995     3096.0
999996     3094.0
999997     3088.0
999998     3068.0
999999     3029.0
1000000    2953.0
1000001    2881.0
Name: streams, dtype: float64


In [8]:
saveVal = 10000 #how much record save in one loop
num = 10 #how many loops
for i in range(0, num):
    dataPart = data.loc[i*saveVal:(i+1)*saveVal-1].copy()
    # Chart
    df_chart = dataPart[['rank', 'title', 'date', 'region', 'chart', 'trend', 'streams']].drop_duplicates().reset_index().drop(columns = ['index'])
    df_chart = df_chart.rename(columns = {'title':'song_id', 'rank':'position', 'date':'day_id', 'region':'region_id', 'chart':'category_id', 'trend':'trend_id'})
    df_chart.insert(0, 'chart_id', range(start + i*saveVal + 1, start + i*saveVal + 1 + len(df_chart)))
    df_chart.replace({"song_id": songDict}, inplace=True) 
    df_chart.replace({"day_id": dayDict}, inplace=True) 
    df_chart.replace({"region_id": regionDict}, inplace=True) 
    df_chart.replace({"category_id": categoryDict}, inplace=True) 
    df_chart.replace({"trend_id": trendDict}, inplace=True)
    df_chart['streams'] = df_chart['streams'].astype(pd.Int64Dtype())

    df_chart.to_csv('../data/df_chart.csv', mode = 'a', header = False, index = False)
